## Homework 3


First of all, we import all the libraries required to make our code work.


In [1]:
import csv
import pandas as pd
import numpy as np
import nltk
import json
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer 
import os
from collections import Counter
import heapq
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import seaborn as sns
import folium
from folium.plugins import MarkerCluster


## Reading CSV, splitting all the rows to tsv

As required in the homework, we created .tsv documents for every row of the Airbnb DataFrame. We used the standard function writer but we had to include the encoding parameter as well.

counter = 0
with open('Airbnb_Texas_Rentals.csv', 'r',encoding="utf8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row: # if row is not empty, write a file with this row
            filename = "doc/doc_%s.tsv" % str(counter)
            with open(filename, 'r',encoding="utf8") as csvfile_out:
                writer = csv.writer(csvfile_out,delimiter="\t")
                writer.writerow(row[1:])
                counter += 1

### Text cleaning

The following task of our homework wanted us to clean the data contained in the tsv files. FIrst of all, to get the total number of documents in the folder we used listdir, a function of the os package. Then we defined functions to go through all the docs in the folder to scan and clean them. We removed punctuation, stemmed words and we lowered case every character. 

In [2]:


DIR = '/Users/canta/Desktop/HMW3/doc/'
ndocs= (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

words=[]
def cleandata(doc):
    txt = doc.read().split('\t')    
    description = txt[4]
    title = txt[7]
    description = description + " " + title    
    #we replace the new lines "tags"
    description = description.replace('\\r', ' ')
    description = description.replace('\\n', ' ')    
    #tokenize the document, strips out the punctuation
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens_d = tokenizer.tokenize(description)
    #lower case of every word
    tokens_d = [w.lower() for w in tokens_d]   
    #stemming
    ps = PorterStemmer()
    tokens_d = [ps.stem(w) for w in tokens_d]   
    #remove stopwords
    stop = set(stopwords.words('english'))
    tokens_d = [w for w in tokens_d if not w in stop]   
    #remove duplicates
    tokens_d = set(tokens_d)
    #append all words we found in words list
    for word in tokens_d:
        words.append(word)
    



In [3]:
for i in range (1,ndocs): #LATER WILL BECOME NDOCS 

    myfile = open('/Users/canta/Desktop/HMW3/doc/doc_'+str(i)+'.tsv', 'r',encoding="utf8")
    cleandata(myfile)


KeyboardInterrupt: 

## Search Engine

In [ ]:
#creating vocabulary - a dictionary with words as keys and integers as IDs

vocabulary = {}
count = 1
for word in words:
    vocabulary[word] = count
    count += 1

## Inverted Index dictionary

In [ ]:
#INVERTED INDEX and docs dictionary: a nested dictionary to retrieve the frequencies for every word in each doc

invIndex = {}
docs = {}

for i in range (1,ndocs): 

    myfile = open('/Users/canta/Desktop/HMW3/doc/doc_'+str(i)+'.tsv', 'r',encoding="utf8")
    filename = os.path.basename((myfile.name).replace('.tsv', ''))
    txt = myfile.read().split('\t')    
    #very common code with the first part - we should build a function in some way
    description = txt[4]
    title = txt[7]
    description = description + " " + title    
    description = description.replace('\\r', ' ')
    description = description.replace('\\n', ' ')
      
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens_inv = tokenizer.tokenize(description)
    
    
    #computed here, the length takes into account the stopwords and excludes the punctuation
    nparole = len(tokens_inv) #compute here to count ...?
    
    
    #lower case 
    tokens_inv = [w.lower() for w in tokens_inv]
   
    #stemming
    ps = PorterStemmer()
    tokens_inv = [ps.stem(w) for w in tokens_inv]
    
    #stopwords
    stop = set(stopwords.words('english'))
    tokens_inv = [w for w in tokens_inv if not w in stop]
    
    #the counter allows us to create a dict with every word as a key and the number of times it appears in the doc
    # as value
    c = Counter(tokens_inv)
    
    c = dict(c)
    #to compute easily the tf for each doc we put into the dictionary a key containing the lenght
    c['LUNGHEZZA2018'] =  nparole
    docs[filename] = c
    
    tokens_inv = set(tokens_inv)
  
    #mechanism to build the inverted index dict
    for word in tokens_inv:
        
        
        #for every word: if it is the first time it appears, create a new key in the dict and append the name of
        # the document in which it is present. if it's already in the dict, append the name of the doc
        if(word in vocabulary.keys()):
            
            if (vocabulary[word]) in invIndex:
                invIndex[vocabulary[word]].append(filename)
            else:
                invIndex[vocabulary[word]] = [filename]
            
    
    

## Query

In [ ]:
#reading the query as input
query = str(input())


In [ ]:
#retrieve the word IDs of the query

#manipulate the text of the query: stemming, lower case, strip punctuation and stopwords, remove duplicates

query = query.split()
ps = PorterStemmer()
query = [ps.stem(w) for w in query]
query = [w.lower() for w in query]
stop = set(stopwords.words('english'))
query = [w for w in query if not w in stop]
query = set(query)

#look at vocabulary dict: from the key of the dict (word) we get the IDs
ids = []
for word in query:
    if word in vocabulary.keys():
        ids.append(vocabulary[word])



In [ ]:
#we have the IDs, now let's get the list of documents in which they're in
querydict = {}
for id in ids:
    querydict[id] = invIndex[id]
    

In [ ]:
#we need only the docs containing all the words, so the point here is to make the intersection between all the lists
#of docs

querylist = []

for key in querydict.keys():
    querylist.append(querydict[key])

#intersection
result = set(querylist[0]).intersection(*querylist)

In [ ]:
def cleanres(result):
    goodres=[]
    urllist=[]
    for each in result:
                myfiles=open('/Users/canta/Desktop/HMW3/doc/'+str(each)+'.tsv','r', encoding="utf8")
                txt = myfiles.read().split('\t')                
                if txt[7] not in urllist: 
                    goodres.append(each)
                    urllist.append(txt[7])
                
    return goodres


In [ ]:
result=cleanres(result)

## Showing the results of the query

In [ ]:
#SHOWING THE RESULTS OF THE QUERY
def showres(result):
    title=[]
    descr=[]
    city=[]
    url=[]
    for each in result:
        myfiles=open('/Users/canta/Desktop/HMW3/doc/'+str(each)+'.tsv','r', encoding="utf8")
        txt = myfiles.read().split('\t')
        title.append(txt[7])
        descr.append(txt[4])
        city.append(txt[2])
        url.append(txt[8])

    data_tuples = list(zip(title,descr,city,url))
    table=pd.DataFrame(data_tuples,columns = ["Title", "Description", "City", "Link"])
    pd.set_option('display.max_colwidth', -1)
    return table

In [ ]:
table=showres(result)
table

# PART 3.2 - tfIdf Inverted Index

In [ ]:
#NEW Inverted Index, includes tfIdf scores

import math

tfidfIdx = {}

for key in invIndex: #here key is the wordID - so: for every word
    
    idf_den = len(invIndex[key]) #n° of different docs containing the word
    
    realWord = (list(vocabulary.keys())[list(vocabulary.values()).index(key)]) #getting word from its ID
    
    for each in invIndex[key]: #each is the doc_i, for the word, take all documents in which it appears
        
        #accessing docs dict by two keys: name of doc and word. get the count and divide it by doc length
        tf = docs[each][realWord]/(docs[each] ['LUNGHEZZA2018']) 
        tfidf = tf * math.log10(ndocs/idf_den)
        
        if(key in tfidfIdx.keys()):
            tfidfIdx[key].append((each, tfidf))
        else:
            tfidfIdx[key] = [(each, tfidf)]
            
        

# Cosine similarity scoring

In [ ]:
#shortcuts to compute the value inside the function below

def eucl_norm(a):
    return math.sqrt(np.dot(a, a))

def cosine_similarity(a, b, c):
    return np.dot(a,b) / (eucl_norm(c) * eucl_norm(b))


In [ ]:
def cosine(doc):
    DOCUMENTO = str(doc)
    myfile = open('/Users/canta/Desktop/HMW3/doc/'+str(doc)+'.tsv', 'r',encoding="utf8")
    txt = myfile.read().split('\t')
    description = txt[4]
    title = txt[7]
    description = description + " " + title
    description = description.replace('\\r', ' ')
    description = description.replace('\\n', ' ')
    tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens_d =tokenizer.tokenize(description)
    tokens_d = [w.lower() for w in tokens_d]
    ps = PorterStemmer()
    tokens_d = [ps.stem(w) for w in tokens_d]
    stop = set(stopwords.words('english'))
    tokens_d = [w for w in tokens_d if not w in stop]
    tokens_d = set(tokens_d)
    
    #entire document
    docwordId =[] #all word IDs of document

    for word in tokens_d: #word in tokens_d is the actual word in the doc
        docwordId.append(vocabulary[word]) #we look into vocabulary to get the IDs   
    
    #we are getting tfidf scores from tfidfIdx for every word id in the doc    
    docsQuery = []
    scores = [] #vector of tfidf scores for all words in the doc:   ||d||
    for idx in docwordId: #for every ID - always all words in doc
        for each in tfidfIdx[idx]: #give me its score, given that we're analyzing one document per time
            if(each[0] == DOCUMENTO):
                scores.append(each[1]) #each[1] is the score of that word in the doc
                if(idx in ids):
                    docsQuery.append(each[1])
                
    #query tfidf  |q| and ||q||
    querytfidf = []
    for ID in ids:
        idf_den = len(invIndex[ID])
        tf = 1/len(ids)
        idf = math.log10(ndocs/idf_den)
        querytfidf.append(tf*idf)
    
    
    a=cosine_similarity(docsQuery,querytfidf, scores)
    return (a,doc)
    
    

In [ ]:
for i in result:
    print(cosine(i))

## Heap Data Structure

According to the homework requests, we used an [heap data structure](https://www.geeksforgeeks.org/heap-data-structure/) to handle the documents resulting. 

In [ ]:
H = []
K=10  #LENGTH WE HAVE DECIDED 
heapq.heapify(H)
for each in result:
    if len(H)<K:
        heapq.heappush(H,cosine(each))
    else:
        a,b=cosine(each)
        heapq.heapreplace(H,cosine(each))
# Use heapify to rearrange the elements
docli=[]
punt=[]
heapq.heapify(H) 
for each in H:
    x,y=each
    docli.append(y)
    punt.append(x)
    
   

table=showres(docli)
table['Scores']=punt
table.sort_values("Scores",ascending=False)

## DEFINE A NEW SCORING FUNCTION


We have defined three new functions: one for the distance index, one for the price index and one for final new scoring function(plus one nice graph to show the distribution for the prices per bedrooms). 

## definedistance

So, the starting point is our set **result**. This set contains all the documents in which the query is contained. We decide to compute the distance of every documents from the city centre of its own city centre. We have done that usign the Geopymodule. We have also normalized our values by dividing by the maxium distance found in one of the documents of result. Here is the formula:

$$\frac{distance of the single document}{maximum distance of one document in result}$$

The outuput of this formula is a list with all the distances normalized.

## defprice

Using the same starting point(result=set containing all the documents in which the query is contained), we decided to use the variables 'average_rate_per_night' and 'bed_room_count' to calculate our final scoring function. First of all, we computed the 'price per bedroom' variable with:

$$listpricesperbed=\frac{averageratepernight}{bedroomcount}$$

we assigned 'bed_room_count'==0 if its values was 'Studio'. At this point we decided to use the quantile distribution of this new variable we have created and we opted to boost all the documents appearing in the second quartile(Practically speaking, the ones that were below the median(low price per bedroom). We usand this kind of weighting process:

 -listpricesperbed[doc]*0.25 if it was contained in the **first quartile**
 
 -listpricesperbed[doc]*0.40 if it was contained in the **second quartile**

 -listpricesperbed[doc]*0.25 if it was contained in the **third quartile**

 -listpricesperbed[doc]*0.10 if it was contained in the **fourth quartile**
 
 Obviously the sum of the weights equals to 1. And finally, we divided each element by the maximum contained in that list.
 
 This fucntion returns a list of prices per bedrooms with all the weighting process applied to it. 
 
 ## def scoringfunction
 
 As our last function we created the new final scoring function. This takes in every document in result with its distance and its price(given by the previous functions). We used this formula to obtain our final scoring function:
 
 $$newscore==(Weightforprice)*(1-pricesperbed)+(weightfordistance)(1-distance)$$
 
 THe weights for both the distance and the price can be decided arbitrarily and we decided to weight more the distance than the prices. The correspondant weights are: 
 $$Weightforprice==0.40$$
 
 $$Weightforprice==0.60$$
 
 This function the returns a value to rank our documents with and the name of the documents which refers to.
 
 
 
 
 **P.S.**: We know we could have written and called everything in a single function but we decided to split the funcitons in order to make everything more understandable (Hopefully we did succeed).

In [ ]:
def definedistance(result):
    distances=[]
    for each in result:
        myfiles=open('/Users/canta/Desktop/HMW3/doc/'+str(each)+'.tsv','r', encoding="utf8")
        doc = myfiles.read().split("\t")
        city =doc[2].lower()
        long=doc[6]
        lat=doc[5]
        geolocator = Nominatim(user_agent="mb") #bo
        locationQ = ((geolocator.geocode(city)).latitude, (geolocator.geocode(city)).longitude) # we should put the city of the query here
        loc = (lat,long) #coordinates retrieved by the doc, pretty easy - to be substituted with above
        distance = geodesic(locationQ, loc).miles
        distances.append(distance)
    mass=max(distances)
    finaldist = [x / mass for x in distances]
    return (list(finaldist))



In [ ]:
def defprice(result):
    listpricesperbed =[]
    for each in result:
        myfiles=open('/Users/canta/Desktop/HMW3/doc/'+str(each)+'.tsv','r', encoding="utf8")
        doc = myfiles.read().split("\t")
        price=doc[0]
        bedrooms=(doc[1])
        price=price.replace("$","")
        if(bedrooms=='Studio'):
            listpricesperbed.append(0)
            continue          
        listpricesperbed.append(int(price)/int(bedrooms))
    global finallist
    finallist=np.array(listpricesperbed)
    firstp=np.percentile(finallist,25)
    secondp=np.percentile(finallist,50)
    thirdp=np.percentile(finallist,75)
    fourthp=np.percentile(finallist,100)
    for i in range(len(finallist)):
        if finallist[i]<firstp:
            finallist[i]=finallist[i]*0.25
        elif finallist[i]<=secondp and finallist[i]>firstp :
            finallist[i]=finallist[i]*0.40
        elif finallist[i]<=thirdp and finallist[i]>secondp :
            finallist[i]=finallist[i]*0.25
        elif finallist[i]<=fourthp and finallist[i]>thirdp:
            finallist[i]=finallist[i]*0.10 
    lista=finallist/(max(finallist))
    return(list(lista))


In [ ]:
def scoringfunction(doc,distanza,price):
    newscore=(0.40*(1-price))+(0.60*(1-distanza))
    return (newscore,doc)

## Heap Data Structure

According to the homework requests, we used an [heap data structure](https://www.geeksforgeeks.org/heap-data-structure/) to handle the documents resulting. 

In [ ]:
distan=definedistance(result)  
pricesperbeds=defprice(result)
H = []
K=10  #LENGTH WE HAVE DECIDED 
heapq.heapify(H)
i=0
for each in result:
    if len(H)<K:
        heapq.heappush(H,scoringfunction(each,distan[i],pricesperbeds[i]))
    else:
        heapq.heapreplace(H,scoringfunction(each,distan[i],pricesperbeds[i]))
    i+=1
# Use heapify to rearrange the elements
docli=[]
punt=[]
heapq.heapify(H) 
for each in H:
    x,y=each
    docli.append(y)
    punt.append(x)
    
   

table=showres(docli)
table['New Scores']=punt
table.sort_values("New Scores",ascending=False)

In [ ]:
import seaborn as sns
sns.kdeplot(finallist, shade=True,)


In [ ]:
sns.boxplot(y=finallist )


## MAP


In [ ]:
def map_func():
    
    geolocator = Nominatim(user_agent="Texas")
    
    text = input("Please enter either the 'place' or the 'coordinates' of your choice [place/coordinates]: ")
    
    
    
    if text == "place":
        place = input("Type the name of the place you are searching for: ")
        rad = int(input("Enter the distance in km: "))
        location = geolocator.geocode(place) 
        #This gives the coordinates of the place
        loc_input = (location.latitude, location.longitude)
    elif text == "coordinates":
        lat = float(input("Enter the latitude: "))
        long = float(input("Enter the longitute: "))
        loc_input = (lat, long)
        #This gives the address of the coordinates
        location = geolocator.reverse(loc_input) 
        print("The coordinates you entered belongs to this address: " + location.address) 

        rad = int(input("Enter the distance in km: "))
        
    else:
        return print("Incorrect input! Enter only 'place' or 'coordinates'") 
    
    tooltip = "Click here"
    #Creating main map
    main_map = folium.Map( 
    location = loc_input,
    zoom_start = 12
    )
    rad = rad*1000 
    #creating s circle within the prescribed distance
    folium.Circle( 
    location = loc_input, 
    radius = rad,
    color = '#3186cc',
    fill = True,
    fill_color = '#3186cc'
    #fill_opacity: 0.6
    ).add_to(main_map)

    #Addning markers
    folium.Marker(location = loc_input, icon = folium.Icon(color='red')).add_to(main_map) 
    
    #Create clusters of the points
    mark_cluster = MarkerCluster().add_to(main_map)
    
    #Creating markers for each location in dataframe
    for i, x in info.iterrows():
        point = (x["latitude"], x["longitude"])
        geography = geodesic(point, loc_input).meters
        
        
        if geography <= rad:
            folium.Marker(location = (x["latitude"], x["longitude"]),
                          popup = folium.Popup('<a href="'+x["url"]+'"'+'target="_blank"> [Click the url for more information] </a>'),
                          tooltip = tooltip
                         ).add_to(mark_cluster)
    
    return main_map